In [1]:
from controllers.pidff import Controller
from tinyphysics import run_rollout_controller
from skopt import gp_minimize
from skopt.space import Real, Integer
import numpy as np


c:\Users\pmazu\Documents\GitHub\controls_challenge\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def optimization_function(params):
    controller = Controller(params)
    data_paths = [f'./data/{i:05}.csv' for i in [0]]
    stored_costs = []
    def rollout(data_paths):
        costs = []
        for data_path in data_paths:
            roll = run_rollout_controller(data_path, controller=controller, model_path='./models/tinyphysics.onnx')
            costs.append(roll[0]['total_cost'])
            if (roll[0]['total_cost'] > 200):
                print("High error on ", data_path)
        cost = np.mean(costs)
        return cost
    last_cost = rollout(data_paths)
    stored_costs.append(last_cost)
    if (last_cost <= 100):
        print("params", params)
        data_paths = [f'./data/{i:05}.csv' for i in range(5, 7)]
        last_cost = rollout(data_paths)
        stored_costs.append(last_cost) 
    return np.mean(stored_costs)


In [3]:
from controllers.samfc import Controller

In [4]:
        # self.v_x0 = params[0]
        # self.alpha_0 = params[1]
        # self.K_alpha = params[2]
        # self.K_p = params[3]
        # self.K_d = params[4]
space = [
    Integer(0, 50, name='v_x0'),
    Real(0.0000001, 10000, name='alpha_0'),
    Real(0, 1, name='K_alpha'),
    Real(0.0, 1000, name='K_p'),
    Real(-1, 1000, name='K_d'),
    # Real(1, 10, name='R_s'),
    # Real(0, 100000, name='L'),
    # Integer(1, 50, name='d_k'),
]

init = [50, 942.4608530709091, 1.0, 314.1324884811456, -1.0]
gp_minimize(optimization_function, space, n_calls=300, random_state=42, verbose=True, x0=init)

Iteration No: 1 started. Evaluating function at provided point.
High error on  ./data/00000.csv
Iteration No: 1 ended. Evaluation done at provided point.
Time taken: 0.6160
Function value obtained: 56035.1362
Current minimum: 56035.1362
Iteration No: 2 started. Evaluating function at random point.
High error on  ./data/00000.csv
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.5686
Function value obtained: 684.4020
Current minimum: 684.4020
Iteration No: 3 started. Evaluating function at random point.
High error on  ./data/00000.csv
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.5801
Function value obtained: 312.5947
Current minimum: 312.5947
Iteration No: 4 started. Evaluating function at random point.
High error on  ./data/00000.csv
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.5651
Function value obtained: 331.2770
Current minimum: 312.5947
Iteration No: 5 started. Evaluating function at random point.
High error on 

          fun: 139.00765388238025
            x: [50, 942.4608530709091, 1.0, 314.1324884811456, -1.0]
    func_vals: [ 5.604e+04  6.844e+02 ...  1.987e+02  1.640e+02]
      x_iters: [[0.0, 0.001, 0, 0, 0], [40, 1834.3478987432948, 0.7796910002727695, 596.8501579464871, 445.27858560644484], [5, 4592.488919712748, 0.3337086111390219, 142.86681792194082, 650.5393614218018], [3, 7219.987722696048, 0.9385527090157504, 0.7787658410143286, 992.2037708505089], [31, 6116.531604921644, 0.007066305219717408, 23.06242504141576, 524.2994349186476], [20, 466.6566322314878, 0.9737555188414594, 232.7713404303043, 89.69704096735363], [31, 3824.6199127333816, 0.9832308858067884, 466.76289324798006, 859.8003471430571], [34, 4504.992519750381, 0.013264961159866532, 942.201755684853, 562.8515060633849], [19, 159.66252230054533, 0.23089382562214905, 241.02546602601177, 682.9467823442837], [30, 8331.949117378326, 0.1733646535077721, 391.06060757324093, 181.4183238758504], [38, 4251.558744969932, 0.207941662